## Case를 나눠서 어떤 Case에서 모델이 좋을지 확인

## Dataset 불러오기

In [1]:
def initialize_resource_paths():
    import os

    # 코랩 mount
    from google.colab import drive
    drive.mount('/content/drive')

    root_directory = '/content/drive/MyDrive/Dataset/'


    """
    원하는 데이터셋은 여기에 추가시키면 돼요
    파일 3개가 뭔차인지 몰라서 이렇게...
    """
    output = dict()
    output['dataset1'] = os.path.join(root_directory, '(자동차보험) 고객별 사고 발생률 예측 모델링_1.csv')
    output['dataset2'] = os.path.join(root_directory, '(자동차보험) 고객별 사고 발생률 예측 모델링_2.csv')
    output['dataset3'] = os.path.join(root_directory, '(자동차보험) 고객별 사고 발생률 예측 모델링_3.csv')


    return output



In [2]:
resource_paths = initialize_resource_paths()

Mounted at /content/drive


In [3]:
def read_user_enriched(input):
    import pandas as pd

    output = pd.read_csv(input, sep=',', encoding = "cp949", engine='python')

    return output

In [7]:
# 불러온 데이터 용량 확인
# 용량이 적당히 크면 잘 불러왔다고  생각가능.
dataset1 = read_user_enriched(resource_paths['dataset1'])
dataset2 = read_user_enriched(resource_paths['dataset2'])
dataset3 = read_user_enriched(resource_paths['dataset3'])
# display('RAM memory usage: {} bytes'.format(datasets.memory_usage().sum()))



In [8]:
import pandas as pd

In [9]:
## 1번째 데이터는 행과 열이 엑셀 전체값이 들어가있어 null 값제거를 한번 해줬다.

df1 = dataset1.iloc[:, :15].dropna()
df1.shape

(105736, 15)

In [10]:
dataset2.shape

(102726, 15)

In [11]:
dataset3.shape

(59234, 15)

In [12]:
# 컬럼 다 합치기

df = pd.concat([df1, dataset2, dataset3], ignore_index=True)
df['YUHO'] = df['YUHO'].str.replace(',', '').astype(int)
df['SAGO'] = df['SAGO'].apply(lambda x: int(float(str(x).replace(',', '').split('.')[0])))
df['SAGO'] = df['SAGO'].replace(',', '').astype(int)



# 변수 이름 변경
df.rename(columns={
    'ZCPRLCLCD': '자동차',
    'ZINSRDAVL': '연령대',
    'ZIOSEXCD': '성별',
    'ZDPRODSCD': '국산구분코드',
    'NCR': '직전3년간사고건수',
    'ZCARPSGVL': '차량경과년수',
    '차종': '배기량별차종',
    'ZDRVLISCD': '운전자한정특별약관코드',
    'ZDRVLISCD___T': '운전자한정특별약관',
    'ZENTCARCD': '가입경력코드',
    'ZCARISDAM': '차량가입금액',
    'ZIMAGERVL': '영상기록장치특약',
    '마일리지약정거리': '마일리지특약',
    'YUHO': '유효대수',
    'SAGO': '사고건수'
}, inplace=True)


# '사고유무' 열 생성
df['사고유무'] = df['사고건수'].apply(lambda x: 0 if x == 0 else 1)

mapping = {'N': 0, 'D': 1, 'C': 2, 'B': 3, 'Z': '기타', '0':'결측치'}
df['직전3년간사고건수'] = df['직전3년간사고건수'].replace(mapping)
df

,자동차,연령대,성별,국산구분코드,직전3년간사고건수,차량경과년수,배기량별차종,운전자한정특별약관코드,운전자한정특별약관,가입경력코드,차량가입금액,영상기록장치특약,마일리지특약,유효대수,사고건수,사고유무
0,A10,0.0,1.0,1.0,0,10년이상,중형,2.0,가족한정(형제자매제외),2.0,5천만원이하,미가입,15000K,0,0,0
1,A10,0.0,1.0,1.0,기타,5년이하,다목적2종,2.0,가족한정(형제자매제외),1.0,미가입,미가입,미가입,0,0,0
2,A10,0.0,1.0,1.0,기타,10년이하,중형,2.0,가족한정(형제자매제외),1.0,미가입,미가입,미가입,0,0,0
3,A10,0.0,1.0,1.0,기타,5년이하,다목적2종,2.0,가족한정(형제자매제외),1.0,5천만원이하,미가입,미가입,0,0,0
4,A10,0.0,2.0,1.0,0,5년이하,중형,1.0,누구나(기본),4.0,5천만원이하,가입,미가입,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267691,A10,40.0,2.0,1.0,2,10년이상,중형,1.0,누구나(기본),8.0,미가입,미가입,7000K,0,0,0
267692,A10,40.0,2.0,1.0,2,5년이하,소형A,12.0,부부 및 자녀한정,8.0,5천만원이하,가입,15000K,2,0,0
267693,A10,40.0,2.0,1.0,2,10년이하,소형A,4.0,기명피보험자1인한정,8.0,미가입,미가입,미가입,6,1,1
267694,A10,40.0,2.0,1.0,2,5년이하,소형B,1.0,누구나(기본),7.0,5천만원이하,가입,미가입,0,0,0


## Case 나누기

In [13]:
import pandas as pd


# 1. 0~10대만 지우기
def remove_age(df, remove_age_list, flag):
    if flag == 0:
      # '연령대' 열에서 remove_age_list 안에 있는 값을 제거
      age_removed_df = df[~df['연령대'].isin(remove_age_list)]

      return age_removed_df

    else:
      return df



# 2. 연령, 성별 빼고 중복된 정보를 제거
def remove_columns_and_duplicates(df, remove_column_list, flag):

    if flag == 0:
      # '연령'과 '성별' 열 제거
      remove_column_df = df.drop(columns=remove_column_list, inplace=False)

      # 중복된 열 제거
      removed_df = remove_column_df.drop_duplicates(keep=False)

      return removed_df

    else:
      return df


# 3. NCR 0을 N으로 합치기 혹은 날리기
def replace_NCR_nan(df, flag):
  if flag == 0:
    filtered_df = df.copy()
    filtered_df['직전3년간사고건수'] = df['직전3년간사고건수'].replace('결측치', 0)
  else:
    filtered_df = df[df['직전3년간사고건수'] != '결측치']

  return filtered_df


# 4. 국산코드 결측치 -> 날리기
def drop_nan_prod(df):
  return df.dropna(subset=['국산구분코드'], inplace=False)

# 5. 사고율이나 유효대수를 SAGO quatile 기준으로 날리기. 날린다면 얼마나?
def remove_SAGO_outliers(df, q):
  trimmed_df = df[df['사고건수'] <= df['사고건수'].quantile(q)]

  return trimmed_df

# 6. 임직원 한정..?
def remove_executives(df, flag):

  if flag == 0:
    df_filtered = df[df['운전자한정특별약관'] != '임직원한정']
    return df_filtered
  else:
    return df

# 7. 사고율에서 유효 0인 경우 유효 1로 잡기 또는 전부 날리기.
def make_sago_rate(df, value):

  new_df = df.copy()

  if value == 0:
    new_df['사고율'] = new_df.apply(lambda row: 0 if row['유효대수'] == 0 else row['사고건수'] / row['유효대수'], axis=1)
  elif value == 1:
    new_df.loc[new_df['유효대수'] == 0, '유효대수'] = 1
    new_df['사고율'] = new_df['사고건수'] / new_df['유효대수']
  else:
    new_df = df[df['유효대수'] != 0].copy()
    new_df['사고율'] = new_df['사고건수'] / new_df['유효대수']

  return new_df


### 모든 Case에서 최적의 값 추론
(Accuracy, F1 score, MSE, recall)

1. 0, 10대 지운것과 안지운것 -> 2진분류

2. 연령성별 빼고 중복된것 제거, 제거X -> 2진분류

3. NCR 0을 날리기 or N에 합치기 -> 2진분류

4. 국산코드 결측치 날리기 -> ..? 그냥 날림 ㅇㅇ

5. 사고 건수를 Quantile 기준으로 날리기. -> 적절한 수치를 찾아야함

6. 임직원 한정 제거 or 제거 X -> 2진분류

7. 유효대수 0인경우, 사고율을 0으로 잡기, 유효대수를 1로 바꾸기, 전부 날리기. -> 3진분류

-> 총 48개 case + quantile


In [14]:
# 사고건수 quantile 값별로 값.
quantile_values = df['사고건수'].quantile([0.70,0.71, 0.86, 0.91,0.95, 0.97, 0.99, 1])

print(quantile_values)

0.70       0.0
0.71       1.0
0.86       2.0
0.91       3.0
0.95       5.0
0.97       9.0
0.99      26.0
1.00    1847.0
Name: 사고건수, dtype: float64


In [15]:
remove_age_list = [0, 10]
remove_column_list = ['연령대', '성별']
quantile_list = [0.70,0.71, 0.86, 0.91,0.95, 0.97, 0.99, 1]

In [16]:
import numpy as np
from tqdm import tqdm
import time
from sklearn.metrics import mean_squared_error




def test_all_case(df0, model_object, X_train, y_train, X_test, y_test):
  sizes = (2, 2, 2, 8, 2, 3)
  df_list = np.zeros(sizes)

  start_time = time.time()

  count = - 1 / 32
  for a in range(2): # 1. 0, 10대 지운것과 안지운것 -> 2진분류
    for b in range(2): # 2. 연령성별 빼고 중복된것 제거, 제거X -> 2진분류
      for c in range(2): # 3. NCR 0을 날리기 or N에 합치기 -> 2진분류
        for e in range(8): # 5. 사고 건수를 Quantile 기준으로 날리기. -> 적절한 수치를 찾아야함
          if (e % 2 == 0):
            count += 1/32
            print(count, "% 완료, 걸린시간:", time.time() - start_time, "초")
          for f in range(2): # 6. 임직원 한정 제거 or 제거 X -> 2진분류
              for g in range(3): # 7. 유효대수 0인경우, 사고율을 0으로 잡기, 유효대수를 1로 바꾸기, 전부 날리기. -> 3진분류
              try:
                dfa = remove_age(df0, remove_age_list, a)
                dfb = remove_columns_and_duplicates(dfa, remove_column_list, b)
                dfc = replace_NCR_nan(dfb, c)
                dfe = remove_SAGO_outliers(dfc, quantile_list[e])
                dff = remove_executives(dfe, f)
                dfg = make_sago_rate(dff, g)

                model = model_object
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)
                mse = mean_squared_error(y_test, y_pred)

                print(mse)
              except Exception as e:






SyntaxError: incomplete input (<ipython-input-16-58c653c2e205>, line 44)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor


# 0. 자동차 구분 제거.
df0 = df.drop('자동차', axis=1)

# 4. 국산코드 결측치 날리기 -> ..? 그냥 날림 ㅇㅇ
dfd = drop_nan_prod(df0)




dfd['사고율'] = dfd.apply(lambda row: 0 if row['유효대수'] == 0 else row['사고건수'] / row['유효대수'], axis=1)

y = dfd[['사고율']]
X = dfd.drop(['사고율', '사고건수', '사고유무'], axis = 1)



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor()
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# MSE 계산
mse = mean_squared_error(y_test, y_pred)

print(mse)

# test_all_case(dfd, model_object, X_train, y_train, X_test, y_test)

<ipython-input-86-ce741a05ef6f>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfd['사고율'] = dfd.apply(lambda row: 0 if row['유효대수'] == 0 else row['사고건수'] / row['유효대수'], axis=1)


ValueError: could not convert string to float: '기타'

In [ ]:
import numpy as np
sizes = (2, 2, 2, 7, 2, 3)
df_list = np.zeros(sizes)
df_list

array([[[[[[0., 0., 0.],
           [0., 0., 0.]],

          [[0., 0., 0.],
           [0., 0., 0.]],

          [[0., 0., 0.],
           [0., 0., 0.]],

          [[0., 0., 0.],
           [0., 0., 0.]],

          [[0., 0., 0.],
           [0., 0., 0.]],

          [[0., 0., 0.],
           [0., 0., 0.]],

          [[0., 0., 0.],
           [0., 0., 0.]]],


         [[[0., 0., 0.],
           [0., 0., 0.]],

          [[0., 0., 0.],
           [0., 0., 0.]],

          [[0., 0., 0.],
           [0., 0., 0.]],

          [[0., 0., 0.],
           [0., 0., 0.]],

          [[0., 0., 0.],
           [0., 0., 0.]],

          [[0., 0., 0.],
           [0., 0., 0.]],

          [[0., 0., 0.],
           [0., 0., 0.]]]],



        [[[[0., 0., 0.],
           [0., 0., 0.]],

          [[0., 0., 0.],
           [0., 0., 0.]],

          [[0., 0., 0.],
           [0., 0., 0.]],

          [[0., 0., 0.],
           [0., 0., 0.]],

          [[0., 0., 0.],
           [0., 0., 0.]],

      

### 아이디어
- 유효대수0을 1로 둠 -----------> 버림이라서 0인데, 올림할거면 다 올림하는게 낫지않나?